In [24]:
import numpy as np
import sklearn
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

In [25]:
l = 2000
n = 2
C = 100

In [26]:

X1 = np.random.multivariate_normal([0,0], [[1,0.5],[0.5,1]], 1000)
X2 = np.random.multivariate_normal([2,2], [[1,-0.5],[-0.5,1]], 1000)
X = np.concatenate([X1, X2])
y = np.array([-1]*1000 + [1]*1000).reshape(l,1)

idx = np.arange(l)
np.random.shuffle(idx)
X = X[idx]
y = y[idx]

X = np.vstack([X.T, np.ones(l)]).T

In [27]:
fig = go.Figure([
    go.Scatter(x=X[:,0], y=X[:,1], marker_color=y[:,0], mode='markers', marker_opacity=0.5),
])
fig.update_layout(
    xaxis=dict(scaleanchor='y', scaleratio=1)
)

In [28]:
# inistial alpha
alpha = np.zeros([l,1])
# alpha = np.random.rand(l,1)
alpha

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [29]:
# initial w
w = np.sum(y * alpha * X, axis=0).reshape(n+1,1)
w

array([[0.],
       [0.],
       [0.]])

In [30]:
Q = (y @ y.T) * (X @ X.T)
Q.shape

(2000, 2000)

In [32]:
U = np.inf
max_iter = 1000
tol = 1e-4

for epoch in tqdm(range(max_iter)):
    _w = w.copy()
    L = np.arange(l)
    np.random.shuffle(L)
    for i in L:
        _alpha = alpha.copy()
        G = y[i] * w.T @ X[i] - 1 + alpha[i] / (2 * C)   # L2 case

        if alpha[i] == 0:
            PG = min(G, 0)
        elif alpha[i] == U:
            PG = max(0, G)
        else:
            PG = G

        if PG != 0:
            alpha[i] = min(max((alpha[i] - G/Q[i][i])[0], 0), U)
            w = np.sum(y * alpha * X, axis=0).reshape(n+1,1)
    
    if np.abs(_w - w).sum() < tol:
        print(f'Algorithm converged in {epoch} steps')
        break

w, alpha, np.mean(alpha == 0)

  0%|          | 0/1000 [00:00<?, ?it/s]

(array([[ 0.84051057],
        [ 0.47383972],
        [-1.76255705]]),
 array([[160.98601503],
        [  0.        ],
        [  0.        ],
        ...,
        [  0.        ],
        [  0.        ],
        [ 29.18483925]]),
 0.5435)

In [33]:
pred = (X @ w > 0).flatten().astype(int)
pred[pred == 0] = -1

conf_matrix = confusion_matrix(y, pred)
np.diag(conf_matrix).sum() / conf_matrix.sum()

0.9

In [34]:
np.mean(alpha==0)

0.5435

In [35]:
x0 = np.linspace(-5, 7, 100)
x1 = np.linspace(-5, 7, 100)

xx, yy = np.meshgrid(x0, x1)
XX = np.array([xx.flatten(), yy.flatten(), np.ones(yy.size)]).T
pred = (XX @ w).reshape(x0.shape[0], x1.shape[0])

In [36]:
fig = go.Figure([
    go.Scatter(x=X[:,0], y=X[:,1], marker_color=y[:,0], mode='markers', marker_opacity=0.5),
    go.Contour(x=x0, y=x1, z=pred, colorscale='thermal',
               contours=dict(coloring ='heatmap', showlabels = True, labelfont_color='white'))
])
fig.update_layout(
    xaxis=dict(scaleanchor='y', scaleratio=1), width=700, height=700
)

In [37]:
svc = LinearSVC(C=10, dual=True, penalty='l2', tol=1e-4)
svc.fit(X, y)
svc.coef_

/Users/albertroethel/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Users/albertroethel/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



array([[ 0.65510852,  0.67551725, -0.79571392]])

In [38]:
svc.score(X, y)

0.9265